# 0. Install and Import Dependencies

In [1]:
!pip3 install mediapipe opencv-python
!pip3 install numpy
!pip3 install tensorflow


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [44]:
import numpy as np


def calc_angle(p1, p2, p3):
    a = np.array([p1.x, p1.y,p1.z])
    b = np.array([p2.x, p2.y, p2.z])
    c = np.array([p3.x, p3.y,p3.z])

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.rad2deg(np.arccos(cosine_angle))

    return angle

def calc_dist(p1, p2):
    a = np.array([p1.x, p1.y,p1.z])
    b = np.array([p2.x, p2.y, p2.z])

    squared_dist = np.sum((a-b)**2, axis=0)
    return np.sqrt(squared_dist)

In [4]:
calc_angle((1,2,3),(1,5,2),(4,2,4))

AttributeError: 'tuple' object has no attribute 'x'

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

# 4. Curl Counter

In [40]:
curr_pos = 'down'
reps = 0

def knee_extension(landmarks):

    
    left_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
    right_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]

    left_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
    right_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]

    left_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

    
    
    errors = []
    leg_down = "left"
    angle = 0
    #Everything is flipped
    new_curr_pos = ""

    if calc_angle(right_ankle, right_knee, right_hip) >= 135 and calc_angle(left_ankle, left_knee, left_hip) >= 135:
        return None

    if left_ankle.y < right_ankle.y:
        leg_down = "right"
    
    if leg_down == "right":
        if calc_angle(right_ankle, right_knee, right_hip) < 140:
            errors.append("Don't bend right leg")
        if abs(left_hip.y - left_knee.y) > 0.05:
            errors.append("Keep left knee parallel to hip")
        if calc_angle(left_ankle, left_knee, left_hip) >= 90 and len(errors) == 0:
            new_curr_pos = "up"
        if calc_angle(left_ankle, left_knee, left_hip) <= 70:
            curr_pos = "down"


            
    if leg_down == "left":
        if calc_angle(left_ankle, left_knee, left_hip) < 140:
            errors.append("Don't bend left leg")
        if abs (right_hip.y - right.knee.y) > 0.05:
            errors.append("Keep right knee parallel to hip")
        if calc_angle(right_ankle, right_knee, right_hip) >= 90 and len(errors) == 0:
            new_curr_pos = "up"
        if calc_angle(right_ankle, right_knee, right_hip) <= 70:
            curr_pos = "down"
    
    if new_curr_pos == "up" and curr_pos == "down":
        reps += 1
        curr_pos = "up"
    



    return errors

In [51]:
def dumbell_thrust(landmarks):

    errors = []

    left_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
    right_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]

    left_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
    right_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]

    left_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]

    left_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
    right_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

    left_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]



    left_knee_angle = calc_angle(left_ankle, left_knee, left_hip)
    right_knee_angle = calc_angle(right_ankle, right_knee, right_hip)

    if left_knee_angle < 90 or right_knee_angle < 90:
        errors.append("Don't bend down too much")
    
    if abs(calc_dist(left_ankle, right_ankle) - calc_dist(left_hip, right_hip)) > 0.1:
        errors.append("Keep legs hip width apart")
    
    if abs(calc_dist(left_elbow, right_elbow) - calc_dist(left_shoulder, right_shoulder)) > 0.1:
        errors.append("Keep elbows parallel to shoulders")

    if calc_dist(left_elbow, left_hip) < 0.3 or calc_dist(right_elbow, right_hip) < 0.3:
        errors.append("Don't bend elbows down too much")
    
    
    return errors

In [52]:
import traceback

# Initialize VideoCapture
cap = cv2.VideoCapture(0)

# Setup mediapipe instance
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils



# Main loop
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Recolor image to RGB
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            #errors = knee_extension(landmarks)
    
            knee_angle = calc_angle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value], landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value], landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])# Display angle and feedback on the frame
            cv2.putText(image, f'Info: {dumbell_thrust(landmarks)}', (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 2)
            #cv2.putText(image, f'Reps: {reps}', (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 2)
  




        except:
            print(traceback.format_exc())
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Knee Movement Analysis', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release VideoCapture and close all windows
cap.release()
cv2.destroyAllWindows()
            

I0000 00:00:1707555246.985872       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2


Traceback (most recent call last):
  File "/var/folders/kv/c6n5tz4d50j_h3b5t7c8jn1m0000gn/T/ipykernel_2531/3444643720.py", line 34, in <module>
    landmarks = results.pose_landmarks.landmark
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'landmark'

Traceback (most recent call last):
  File "/var/folders/kv/c6n5tz4d50j_h3b5t7c8jn1m0000gn/T/ipykernel_2531/3444643720.py", line 34, in <module>
    landmarks = results.pose_landmarks.landmark
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'landmark'

Traceback (most recent call last):
  File "/var/folders/kv/c6n5tz4d50j_h3b5t7c8jn1m0000gn/T/ipykernel_2531/3444643720.py", line 34, in <module>
    landmarks = results.pose_landmarks.landmark
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'landmark'

Traceback (most recent call last):
  File "/var/folders/kv/c6n5tz4d50j_h3b5t7c8jn1m0000gn/T

KeyboardInterrupt: 

In [ ]:
errors